In [1]:
from urllib.request import urlopen
import json
import pandas as pd
import re
from collections import defaultdict

## Function that allows us to automate calls to the API so that we can extract data fro Large Data Sets ##


In [ ]:
def get_blast_info(sn_name):
    """
    Fetch information for a supernova/transient from the BLAST API.

    Parameters
    ----------
    sn_name : str
        The supernova/transient name (e.g. "2004ef").

    Returns
    -------
    tuple
        (row_dict, missing_name_or_None)
        - On success: (dict of BLAST fields, None)
        - On failure: ({}, sn_name)
    """

    url = f"https://blast.ncsa.illinois.edu/api/transient/get/{sn_name}?format=json"
    try:
        with urlopen(url) as r:
            payload = json.loads(r.read())
    
    ## The following catches any Python Exception that happened within the "try" block.
    ## Error object gets stored as the variable e.
    ## Also keeps track of missing supernova names. Upon exceptions being thrown, ends up returning just
    ## the string that is the SNIa name. 
    except Exception as e:
        print(f"Error fetching {sn_name}: {e}")
        return {}, sn_name

    row = {
        "blast_transient_spectroscopic_class": (payload.get("transient_spectroscopic_class") or "").strip(),
        "blast_transient_photometric_class": (payload.get("transient_photometric_class") or "").strip(),
        
        "blast_host_name": payload.get("host_name"),
        "blast_host_ra_deg": payload.get("host_ra_deg"),
        "blast_host_dec_deg": payload.get("host_dec_deg"),
        "blast_host_redshift": payload.get("host_redshift"),
        "blast_host_EBV_MW": payload.get("host_milkyway_dust_reddening"),

        # Local SED
        "blast_local_log_mass_50": payload.get("local_aperture_host_log_mass_50"),
        "blast_local_log_sfr_50": payload.get("local_aperture_host_log_sfr_50"),
        "blast_local_log_ssfr_50": payload.get("local_aperture_host_log_ssfr_50"),
        "blast_local_log_age_50": payload.get("local_aperture_host_log_age_50"),
        "blast_local_mass_surviving_ratio": payload.get("local_aperture_host_mass_surviving_ratio"),

        # Global SED
        "blast_global_log_mass_50": payload.get("global_aperture_host_log_mass_50"),
        "blast_global_log_sfr_50": payload.get("global_aperture_host_log_sfr_50"),
        "blast_global_log_ssfr_50": payload.get("global_aperture_host_log_ssfr_50"),
        "blast_global_log_age_50": payload.get("global_aperture_host_log_age_50"),
        "blast_global_mass_surviving_ratio": payload.get("global_aperture_host_mass_surviving_ratio"),
    }
    return row, None

In [ ]:
# Load your CSV 
csv_path = "des_sample_SNIa.csv"
df = pd.read_csv(csv_path)

# Collect BLAST info
blast_rows = []
missing_from_blast = []

for sn in df["CID"]:   # or df["SNID"]
    row, missing = get_blast_info(sn)
    blast_rows.append(row)
    if missing:   # missing is sn_name or None
        missing_from_blast.append(missing)

# Merge results back into DataFrame
blast_df = pd.DataFrame(blast_rows)
df = pd.concat([df, blast_df], axis=1)

# Save to new file
df.to_csv("explore_webapi_with_blast.csv", index=False)

print("Failed lookups:", missing_from_blast)